### 目的：爬取Momo購物網折價品訊息，以便得知目前下哪個商品最划算。
### 流程：首頁 -> 手機 -> iphone -> 折價券熱銷 -> 爬取“折價券熱銷”頁面所有商品 

In [1]:
import requests
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

req = requests.get('https://www.momoshop.com.tw/category/DgrpCategory.jsp?d_code=1905200036&CPHOT=Y&flag=L',headers = headers)
#req #-->此時跳錯：ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
# -->猜測網站可能有防爬機制，以致無法順利爬取內容
# -->只要騙網站說我們是真人，不是爬蟲就好 => 在import下方加上headers,user agent資訊

#req.text   #--->顯示網頁原始碼


In [2]:
#把原始碼丟給BeatuifulSoup解析，把解析結果丟給soup，以取得商品資訊
from bs4 import BeautifulSoup as bs
soup =  bs(req.text,'lxml')
soup

<!DOCTYPE HTML>
<html lang="zh">
<head>
<link href="/main/favicon.ico" rel="shortcut icon"/>
<link href="/main/favicon.ico" rel="Bookmark"/>
<link href="/main/favicon.ico" rel="icon" type="image/ico"/>
<link href="/search/openSearch.xml" rel="search" title="momo購物網" type="application/opensearchdescription+xml"/>
<link href="https://m.momoshop.com.tw/category.momo?cphot=y&amp;cn=1905200036" media="only screen and (max-width: 640px)" rel="alternate"/>
<title>折價券熱銷,館長推薦,iPhone,手機/平板-momo購物網</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="折價券熱銷,館長推薦,iPhone,手機/平板-momo購物網" name="Title"/>
<meta content="momo購物網" name="Author"/>
<meta content="momoshop,momo購物網" name="Subject"/>
<meta content="折價券熱銷,館長推薦,iPhone,手機/平板," name="Description"/>
<meta content="折價券熱銷,館長推薦,iPhone,手機/平板,◤限量優質福利品" name="Keywords"/>
<meta content="Tradition Chinese" name="Language"/>
<meta content="momoshop,momo購物網"

In [3]:
goods = [] # 設定一個空白list，用來裝爬出來的結果

for good in soup.select('.eachGood'): # 從原始碼觀察到商品資訊都放在li class為"eachGood"裡面
#     print(good)
    title = good.select_one('.prdName').get('title')
    price_r = good.select_one('.prdPrice').text.replace('$','').replace(',','')  #爬下結果立即去除多餘的字元
    price = int(price_r)  #放進資料庫前先處理資料型態
    icon = good.select_one('.iconArea').text
    goods.append({'title':title,'price':price,'icon':icon})
#     print('-'*30) # 把所有商品資訊爬下來，並用分隔線區把商品分開來

In [4]:
# 將結果放進df裡
import pandas as pd
df = pd.DataFrame(goods)
df.dtypes

title    object
price     int64
icon     object
dtype: object

In [5]:
# 依價格低到高，排序前10筆資料
df.head(10).sort_values(by = ['price'],ascending = True)

,title,price,icon
7,【Apple 蘋果】福利品 iPhone 7 128GB 4.7吋智慧型手機(8成新),7011,速折價券登記
0,【Apple 蘋果】福利品 iPhone 8 4.7吋 64GB 手機(搭配原廠充電器+原廠...,7458,速折價券登記
4,【Apple 蘋果】福利品 iPhone 7 Plus 5.5寸手機 128GB(電池健康度...,8978,速折價券登記
8,【Apple 蘋果】福利品 iPhone 8 4.7吋64G智慧型手機(全機原廠零件+安心...,9982,速折價券登記
5,【Apple 蘋果】福利品 iPhone 8 Plus 5.5吋手機 64GB(驚爆贈品-氣...,11495,折價券登記
6,【Apple 蘋果】福利品 iPhone 8 Plus 256GB 智慧手機,13661,速折價券登記
2,【Apple 蘋果】福利品 iPhone 8 Plus 5.5吋64G智慧型手機(全機內部...,13918,速折價券登記
1,【Apple 蘋果】福利品 iPhone X 256GB 智慧手機,16141,速折價券登記
3,【Apple 蘋果】福利品 iPhone Xs 64GB,16910,速折價券登記
9,【Apple 蘋果】福利品 iPhone Xs Max 64GB,19950,折價券登記


In [6]:
# 依價格低到高，排序所有爬下的資料
df.sort_values(by = ['price','title'],ascending = True)

,title,price,icon
22,【Apple 蘋果】福利機 iPhone6s Plus 64G(外觀九成新),6151,速折價券登記
12,【Apple 蘋果】福利機 iPhone7 128G(外觀九成新),6500,速折價券登記
20,【Apple 蘋果】福利品 iPhone 7 Plus 5.5吋手機 32GB(搭原廠充電器...,6612,速折價券登記
18,【Apple 蘋果】福利品 iPhone 6s Plus 5.5吋手機 64GB(電池健康度...,6650,速折價券登記
16,【Apple 蘋果】福利品 iPhone 8 64GB 4.7吋 智慧型手機(贈玻璃貼+空壓殼),6992,速折價券登記
7,【Apple 蘋果】福利品 iPhone 7 128GB 4.7吋智慧型手機(8成新),7011,速折價券登記
13,【Apple 蘋果】福利品 iPhone 7 4.7吋手機 128GB(電池健康度100%+...,7220,速折價券登記
0,【Apple 蘋果】福利品 iPhone 8 4.7吋 64GB 手機(搭配原廠充電器+原廠...,7458,速折價券登記
15,【Apple 蘋果】福利品 iPhone 8 64G 4.7寸 9成新,7600,折價券登記
17,【Apple 蘋果】福利品 iPhone 7 Plus 5.5寸手機 32GB(電池健康度1...,7961,速折價券登記


# ===== 其他 =====

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   28 non-null     object
 1   price   28 non-null     int64 
 2   icon    28 non-null     object
dtypes: int64(1), object(2)
memory usage: 800.0+ bytes


### *DataFrame.astype(self: ~FrameOrSeries, dtype, copy: bool = True, errors: str = 'raise') → ~FrameOrSeries
### ---> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html

### *Python DataFrame 欄位型態轉換方法
### ---> https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/358035/